# Goal

The goal of this notebook is to detect and replace the values of any loss of data

## Sensor dataset

We have two types of outliers in out dataset, missing data aka NaN or data putted as 0.
For both temperatures, it's pretty straightforwards because the temps never goes below 20 degrees (roughly), so detecting missing values is easy.

But for irradiation, we have to detect where values at 0 shouldn't be there.

UPDDATE 1: Noticed that I was wrong, no real value at 0 but lots of missing values, not NaN but missing, so need to remake the dataset to add those dates and times at those points.

To do that, we'll "split" the dataset use Kmeans clustering, and detect all values at 0 in the middle of the day.

Afterwards, we'll be using the "cleaned" set to apply feature engineering for each columns, and finally chose, train and use a model to replace each missing/wrong values

Finally, we'll save the new dataset as parquet file.


In [1]:
from src.data_prep import *
from src.feature_engineering import *
# Load both training and dataset used for end completion. 
dfs = load_sensor("dataset/Plant_1_Weather_Sensor_Data.csv", save=False)
# Get the X and y for all expectedd values
irr_X, irr_y = sensor_for_irradiation(dfs[0])
atemp_X, atemp_y = sensor_for_ambient_temp(dfs[0])
mtemp_X, mtemp_y = sensor_for_module_temp(dfs[0])

random_state_nb = 42
test_size = 0.2

irrX_train, irrX_dev, irry_train, irry_dev = train_test_split(irr_X, irr_y, test_size=test_size, random_state=random_state_nb)
atempX_train, atempX_dev, atempy_train, atempy_dev = train_test_split(at_X, at_y, test_size=test_size, random_state=random_state_nb)
mtempX_train, mtempX_dev, mtempy_train, mtempy_dev =  train_test_split(amX, amy, test_size=test_size, random_state=random_state_nb)


Xs = {
    'IRRADIATION': (irrX_train, irrX_dev),
    'AMBIENT_TEMPERATURE': (atempX_train, atempX_dev),
    'MODULE_TEMPERATURE': (mtempX_train, mtempX_dev),
}

ys = {
    'IRRADIATION': (irry_train, irry_dev) ,
    'AMBIENT_TEMPERATURE': (atempy_train, atempy_dev),
    'MODULE_TEMPERATURE': (mtempy_train, mtempy_dev) ,
}

### IRRADIATION (Chosing best models)

As suspecting non-linear relationships, I'll focus on those first models to take a grasp of our goals:
- SVMs
- DecisionTree
- Random Forest
- Gradient Boosting

Additionnal models that can be noted for further tests:
- Neural Nets
- XGBoost
- LightGBM

These metrics will be used to get a good overview of models performances on requested tasks:
- R-Squared
- MAE
- MSE
- RMSE


In [ ]:
# Define and optimize the best model
from src.optimization import *

irr_model = GradientBoostingRegressor()
mtemp_model = GradientBoostingRegressor()
atemp_model = GradientBoostingRegressor()

model_d = {
    'IRRADIATION': irr_model,
    'AMBIENT_TEMPERATURE': mtemp_model,
    'MODULE_TEMPERATURE': atemp_model,
}


for k,v in model_d.items():
    model_d[k] = normgb_opt(Xs[k], ys[k])

In [ ]:
# Applying mdels

changed_df = write_df(dfs[1])

In [4]:

from src.model import * 
mods = []
for m in ['SVM', 'DT', 'RF', 'GB']:
    model, metrics = model_testing(irr_X,irr_y, m)
    mods.append(mods)
    print(f'-----------{m}-----------')
    print(f"r2:{metrics['r2']}\nMAE:{metrics['MAE']}\nMSE:{metrics['MSE']}\nRMSE:{metrics['RMSE']}")  

Fitting 2 folds for each of 40 candidates, totalling 80 fits
{'C': 16, 'gamma': 0.1, 'kernel': 'linear'}
-----------SVM-----------
r2:0.9570077536976176
MAE:0.0392994736688664
MSE:0.0036580574443085536
RMSE:0.060481876990620534
Fitting 2 folds for each of 16 candidates, totalling 32 fits
{'criterion': 'squared_error', 'max_depth': 11, 'min_samples_split': 12, 'splitter': 'random'}
-----------DT-----------
r2:0.9566498064182145
MAE:0.030151802033353967
MSE:0.003701326460104872
RMSE:0.06083852776082663
Fitting 2 folds for each of 12 candidates, totalling 24 fits
{'criterion': 'squared_error', 'max_depth': 2, 'max_features': 'log2', 'min_samples_split': 2, 'n_estimators': 85}
-----------RF-----------
r2:0.9378672041932105
MAE:0.045347806172349986
MSE:0.004697701540228066
RMSE:0.06853978071330595
Fitting 2 folds for each of 24 candidates, totalling 48 fits
{'learning_rate': 0.1, 'max_depth': 1, 'min_samples_split': 2, 'n_estimators': 160, 'subsample': 0.6}
-----------GB-----------
r2:0.960

Without surprise, Gradient boosting is one of the best models and goes toe to toe with randomforest (And slightly better if we take all metrics into consideration).
And quite surprisingly, SVM and Decision Tree are also toe-to-toe but on the lower end.

Now we'll try out adding some shallow tuning to see if there's any differences.

**UPDATE 1:**
Has it look like the computation time for my grids are way too high, reason are combination of many possibilities AND numbers are too high.

**Example with SVR**: Computing C as 100 is twice or even thrices the computing time of 50.

SO i'm putting my expectations way down to get a first line of sight of the parameters and improvement I could do.
For some example of computations time tested (SO far for SVR (sight)) are twice 800 mins and one 1200, each reviewing the grid by nearly halfing it.

**UPDATE 2:**
The SVR computationnal issue was situated on the fact that a poly kernel doesn't fit well with gamma (need to expand on why exactly though)
This was taking ages for nothings.
Now should be resolved.|

**UPDATE 3:**

After simple tuning, we can bost SVM & DT to a way better fitting, for them to be closer to GB & RF.
On the other hands, parameters used for fitting RF are not sufficient or too shallow, hence giving an overall worse prediction.

In [9]:
# Trying out other libraries' 
import xgboost as xgb
import lightgbm as lgb
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

x_train, x_dev, y_train, y_dev = train_test_split(irr_X, irr_y, test_size=0.2, random_state=42)
extreme = xgb.XGBRegressor()
light = lgb.LGBMRegressor()
percep = MLPRegressor()

extreme.fit(x_train, y_train)
light.fit(x_train, y_train)
percep.fit(x_train, y_train)

preds  = []
preds.append(extreme.predict(x_dev))
preds.append(light.predict(x_dev))
preds.append(percep.predict(x_dev))

for index, p in enumerate(preds):
    print(index)
    print(r2_score(p, y_dev))
    print(r2_score(y_dev, p))
    print(mean_absolute_error(p, y_dev))
    print(mean_absolute_error(y_dev, p))
    print(mean_squared_error(y_dev,p, squared=True))
    print(mean_squared_error(y_dev,p, squared=False))

# Finishing by deeply tuning the expected candidate for best fitting.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000325 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1127
[LightGBM] [Info] Number of data points in the train set: 2545, number of used features: 8
[LightGBM] [Info] Start training from score 0.231840
0
0.9496143796207761
0.9500916169818323
0.03206345258992144
0.03206345258992144
0.004268907342379536
0.0653368758235312
1
0.9588535545223675
0.9591986696778222
0.029951370402308206
0.029951370402308206
0.0034899367212076105
0.05907568637948789
2
0.8671365902354395
0.8587794302698107
0.08877957665221778
0.08877957665221778
0.01207928389098027
0.10990579552953643


Conclusion: Plain lightgbm is working nearly as well as shallow tuned GB

#### Final Tuning

For finishing the full tuning process I'll be using random search technique as a part of the last parameter tuning while keeping in mind that the tuning part should be done within a night and/or within additional few hours.

Will do it for Common Gradient Boosting and LightGBM as well.


In [1]:
from sklearn.model_selection import train_test_split
from src.optimization import *
from src.data_prep import *
from src.feature_engineering import *

dfs = load_sensor("dataset/Plant_1_Weather_Sensor_Data.csv", save=False)
irr_X, irr_y = sensor_for_irradiation(dfs[0])

x_train, x_dev, y_train, y_dev = train_test_split(irr_X, irr_y, test_size=0.2, random_state=42)
params, model = normgb_opt(x_train, y_train)

# Getting dependencies issues for numpy and skopt, might need to change back to grid/randomsearch  

AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

#### Testing & Displaying

## Testing if getting the same values 